In [1]:
from pyscipopt import Model, quicksum

# Datos del problema
utilidad = {
    (1,2): 12, (1,3): 22, (1,4): 38, (1,5): 40,
    (2,3): 13, (2,4): 20, (2,5): 29,
    (3,4): 10, (3,5): 19,
    (4,5): 12
}

# Modelo
model = Model("Alquiler")

# Variables: x[i,j] = 1 si el contrato va desde año i hasta j
x = {}
for (i,j) in utilidad.keys():
    x[i,j] = model.addVar(vtype="B", name=f"x_{i}_{j}")

# Objetivo: maximizar la utilidad total
model.setObjective(quicksum(utilidad[i,j]*x[i,j] for (i,j) in utilidad), "maximize")

# Restricciones: cada año 1..4 debe estar cubierto exactamente por un contrato
for t in range(1,5):  # años 1,2,3,4
    model.addCons(
        quicksum(x[i,j] for (i,j) in utilidad if i <= t < j) == 1,
        name=f"cobertura_anio_{t}"
    )

# Optimizar
model.optimize()

# Mostrar resultados
print("Solución óptima:")
for (i,j) in utilidad:
    if model.getVal(x[i,j]) > 0.5:
        print(f"Contrato desde año {i} hasta {j} (utilidad {utilidad[i,j]})")

print("Utilidad total:", model.getObjVal())


Solución óptima:
Contrato desde año 1 hasta 4 (utilidad 38)
Contrato desde año 4 hasta 5 (utilidad 12)
Utilidad total: 50.0


In [2]:
sol = model.getBestSol()

model.writeSol(sol, 'soluciones/ejercicio_6.sol')